In [ ]:
pip install requests beautifulsoup4

In [33]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re
import os
from urllib.parse import urljoin 

# データベースファイル名
DB_NAME = 'github_repos.db'
# スクレイピング対象のURLのベース
BASE_URL = 'https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers'

def create_database():
    """
    SQLiteデータベースを作成し、テーブルを定義します。
    """
    if os.path.exists(DB_NAME):
        print(f"既存のデータベースファイル '{DB_NAME}' を削除します。")
        os.remove(DB_NAME)
        
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()

        cursor.execute('''
            CREATE TABLE repositories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                repo_name TEXT UNIQUE NOT NULL,
                language TEXT,
                stars INTEGER
            )
        ''')
        
        conn.commit()
        print(f"データベース '{DB_NAME}' とテーブル 'repositories' を作成しました。")
        return conn
    except sqlite3.Error as e:
        print(f"データベース作成エラー: {e}")
        return None

def scrape_github_repos(conn):
    """GitHubからリポジトリ情報をスクレイピングしてDBに保存します。（URL生成によるページネーション対応）"""
    
    if conn is None:
        return

    page_number = 1
    total_repos_scraped = 0
    cursor = conn.cursor()
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # ページネーションをURL生成で処理する
    while True:
        # URLを生成 (page=N パラメータを追加/置換)
        if page_number == 1:
            current_url = BASE_URL
        else:
            # ページ番号をURLに追加するシンプルなロジック
            current_url = f"{BASE_URL}&page={page_number}"
        
        print(f"\n--- 🌐 ページ {page_number} をスクレイピング中 ---")
        print(f"URL: {current_url}")
        
        time.sleep(1) # ページアクセス前の遅延

        try:
            response = requests.get(current_url, headers=headers)
            response.raise_for_status() 
        except requests.exceptions.RequestException as e:
            print(f"HTTPリクエストエラー (ページ {page_number}): {e}")
            break 

        soup = BeautifulSoup(response.content, 'html.parser')
        
        # リポジトリ一覧のコンテナ要素を特定（Box-rowを使用）
        repo_list = soup.find_all('li', class_='Box-row')
        
        if not repo_list:
            if page_number == 1:
                print("⚠️ 警告: 1ページ目でリポジトリ要素が全く見つかりません。セレクタを確認してください。")
            else:
                print(f"このページ ({page_number}) でリポジトリが見つかりませんでした。全件取得完了と見なします。")
            break

        print(f"  - このページで {len(repo_list)} 個のリポジトリを発見しました。")

        # --- リポジトリ情報の抽出とDB保存 ---
        repos_on_page = 0
        for i, repo in enumerate(repo_list):
            
            # 1. リポジトリ名の取得ロジックを強化: href="/google/リポジトリ名" のリンクを直接探す
            repo_link = repo.find('a', href=re.compile(r'/google/[^/]+$')) 
            repo_name = repo_link.text.strip() if repo_link else None

            # リポジトリ名が取得できない場合は、処理をスキップ
            if not repo_name:
                continue

            # 2. 主要な言語
            lang_tag = repo.find('span', itemprop='programmingLanguage')
            language = lang_tag.text.strip() if lang_tag else 'N/A'
            
            # 3. スターの数
            # href="/google/repo-name/stargazers" のリンクを持つ <a> タグ
            star_tag = repo.find('a', href=re.compile(f'/{re.escape(repo_name)}/stargazers'))
            stars_text = star_tag.parent.text.strip() if star_tag and star_tag.parent else '0'

            try:
                stars = int(stars_text.replace(',', ''))
            except ValueError:
                stars = 0
                
            try:
                cursor.execute('''
                    INSERT OR REPLACE INTO repositories (repo_name, language, stars) 
                    VALUES (?, ?, ?)
                ''', (repo_name, language, stars))
                total_repos_scraped += 1
                repos_on_page += 1
            except sqlite3.Error as e:
                print(f"DB挿入エラー for {repo_name}: {e}")

            # 課題の要件: time.sleep(1) を必ず入れること (リポジトリごと)
            time.sleep(1)

        conn.commit()
        
        # 10個すべて取得できた場合のみ次のページに進む
        if repos_on_page == 10:
             page_number += 1
        else:
             # 10個未満しか取得できなかった場合、最終ページと見なす
             break
            
    print(f"\nDBへの保存が完了しました。合計 {total_repos_scraped} 個のリポジトリを取得しました。")


def display_data(conn):
    """保存されたデータをSELECT文で表示します。"""
    
    if conn is None:
        return

    print("\n--- 💾 データベース保存データ (SELECT文による表示) ---")
    
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT repo_name, language, stars FROM repositories ORDER BY stars DESC")
        rows = cursor.fetchall()
        
        if not rows:
            print("データがありません。")
            return
            
        # ヘッダー表示
        print(f"{'No.':<4} | {'リポジトリ名':<40} | {'主要な言語':<20} | {'スターの数':>10}")
        print("-" * 80)
        
        # データ表示
        for i, row in enumerate(rows):
            repo_name, language, stars = row
            print(f"{i+1:<4} | {repo_name:<40} | {language:<20} | {stars:>10,}") 

    except sqlite3.Error as e:
        print(f"DB SELECTエラー: {e}")

def main():
    """メイン処理"""
    
    conn = create_database()
    
    if conn:
        print("\n⚠️ 注意: time.sleep(1)がリポジトリごとに適用されるため、処理に数分から十数分かかる可能性があります。")
        
        scrape_github_repos(conn)
        
        display_data(conn)
        
        conn.close()
        print("\n✅ 処理が完了しました。")

if __name__ == "__main__":
    main()

既存のデータベースファイル 'github_repos.db' を削除します。
データベース 'github_repos.db' とテーブル 'repositories' を作成しました。

⚠️ 注意: time.sleep(1)がリポジトリごとに適用されるため、処理に数分から十数分かかる可能性があります。

--- 🌐 ページ 1 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 2 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=2
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 3 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=3
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 4 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=4
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 5 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=5
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 6 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort

KeyboardInterrupt: 

In [22]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re
import os
from urllib.parse import urljoin 

# データベースファイル名
DB_NAME = 'github_repos.db'
# スクレイピング対象のURLのベース
BASE_URL = 'https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers'

def create_database():
    """
    SQLiteデータベースを作成し、テーブルを定義します。
    """
    if os.path.exists(DB_NAME):
        print(f"既存のデータベースファイル '{DB_NAME}' を削除します。")
        os.remove(DB_NAME)
        
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()

        cursor.execute('''
            CREATE TABLE repositories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                repo_name TEXT UNIQUE NOT NULL,
                language TEXT,
                stars INTEGER
            )
        ''')
        
        conn.commit()
        print(f"データベース '{DB_NAME}' とテーブル 'repositories' を作成しました。")
        return conn
    except sqlite3.Error as e:
        print(f"データベース作成エラー: {e}")
        return None

def scrape_github_repos(conn):
    """GitHubからリポジトリ情報をスクレイピングしてDBに保存します。（URL生成によるページネーション対応）"""
    
    if conn is None:
        return

    page_number = 1
    total_repos_scraped = 0
    cursor = conn.cursor()
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # ページネーションをURL生成で処理する
    while True:
        # URLを生成 (page=N パラメータを追加/置換)
        if page_number == 1:
            current_url = BASE_URL
        else:
            # ページ番号をURLに追加するシンプルなロジック
            current_url = f"{BASE_URL}&page={page_number}"
        
        print(f"\n--- 🌐 ページ {page_number} をスクレイピング中 ---")
        print(f"URL: {current_url}")
        
        time.sleep(1) # ページアクセス前の遅延

        try:
            response = requests.get(current_url, headers=headers)
            response.raise_for_status() 
        except requests.exceptions.RequestException as e:
            print(f"HTTPリクエストエラー (ページ {page_number}): {e}")
            break 

        soup = BeautifulSoup(response.content, 'html.parser')
        
        # リポジトリ一覧のコンテナ要素を特定（Box-rowを使用）
        repo_list = soup.find_all('li', class_='Box-row')
        
        if not repo_list:
            if page_number == 1:
                print("⚠️ 警告: 1ページ目でリポジトリ要素が全く見つかりません。セレクタを確認してください。")
            else:
                print(f"このページ ({page_number}) でリポジトリが見つかりませんでした。全件取得完了と見なします。")
            break

        print(f"  - このページで {len(repo_list)} 個のリポジトリを発見しました。")

        # --- リポジトリ情報の抽出とDB保存 ---
        repos_on_page = 0
        for i, repo in enumerate(repo_list):
            
            # 1. リポジトリ名の取得ロジックを強化: href="/google/リポジトリ名" のリンクを直接探す
            repo_link = repo.find('a', href=re.compile(r'/google/[^/]+$')) 
            repo_name = repo_link.text.strip() if repo_link else None

            # リポジトリ名が取得できない場合は、処理をスキップ
            if not repo_name:
                continue

            # 2. 主要な言語
            lang_tag = repo.find('span', itemprop='programmingLanguage')
            language = lang_tag.text.strip() if lang_tag else 'N/A'
            
            # 3. スターの数
            # href="/google/repo-name/stargazers" のリンクを持つ <a> タグ
            star_tag = repo.find('a', href=re.compile(f'/{re.escape(repo_name)}/stargazers'))
            stars_text = star_tag.parent.text.strip() if star_tag and star_tag.parent else '0'

            try:
                stars = int(stars_text.replace(',', ''))
            except ValueError:
                stars = 0
                
            try:
                cursor.execute('''
                    INSERT OR REPLACE INTO repositories (repo_name, language, stars) 
                    VALUES (?, ?, ?)
                ''', (repo_name, language, stars))
                total_repos_scraped += 1
                repos_on_page += 1
            except sqlite3.Error as e:
                print(f"DB挿入エラー for {repo_name}: {e}")

            # 課題の要件: time.sleep(1) を必ず入れること (リポジトリごと)
            time.sleep(1)

        conn.commit()
        
        # 10個すべて取得できた場合のみ次のページに進む
        if repos_on_page == 10:
             page_number += 1
        else:
             # 10個未満しか取得できなかった場合、最終ページと見なす
             break
            
    print(f"\nDBへの保存が完了しました。合計 {total_repos_scraped} 個のリポジトリを取得しました。")


def display_data(conn):
    """保存されたデータをSELECT文で表示します。"""
    
    if conn is None:
        return

    print("\n--- 💾 データベース保存データ (SELECT文による表示) ---")
    
    cursor = conn.cursor()
    
    try:
        cursor.execute("SELECT repo_name, language, stars FROM repositories ORDER BY stars DESC")
        rows = cursor.fetchall()
        
        if not rows:
            print("データがありません。")
            return
            
        # ヘッダー表示
        print(f"{'No.':<4} | {'リポジトリ名':<40} | {'主要な言語':<20} | {'スターの数':>10}")
        print("-" * 80)
        
        # データ表示
        for i, row in enumerate(rows):
            repo_name, language, stars = row
            print(f"{i+1:<4} | {repo_name:<40} | {language:<20} | {stars:>10,}") 

    except sqlite3.Error as e:
        print(f"DB SELECTエラー: {e}")

def main():
    """メイン処理"""
    
    conn = create_database()
    
    if conn:
        print("\n⚠️ 注意: time.sleep(1)がリポジトリごとに適用されるため、処理に数分から十数分かかる可能性があります。")
        
        scrape_github_repos(conn)
        
        display_data(conn)
        
        conn.close()
        print("\n✅ 処理が完了しました。")

if __name__ == "__main__":
    main()

データベース 'github_repos.db' とテーブル 'repositories' を作成しました。

⚠️ 注意: time.sleep(1)がリポジトリごとに適用されるため、処理に数分から十数分かかる可能性があります。

--- 🌐 ページ 1 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 2 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=2
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 3 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=3
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 4 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=4
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 5 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=5
  - このページで 10 個のリポジトリを発見しました。

--- 🌐 ページ 6 をスクレイピング中 ---
URL: https://github.com/google?tab=repositories&q=&type=all&language=&sort=stargazers&page=6
  - このページで 10 個のリポジトリ